In [ ]:
import pandas as pd

In [ ]:
DATA_DIR = 'data/'
DATA_FILE = 'BalanceSPL_Fonction_2018_Dec2019.csv'

In [ ]:
data = pd.read_csv(DATA_DIR + DATA_FILE, sep=';')

In [ ]:
COLUMNS = ['IDENT', 'NDEPT', 'INSEE', 'siren', 'LBUDG', 'COMPTE', 'OOBDEB', 'OOBCRE', 
           'FONCTION', 'SD', 'SC', 'OBNETDEB', 'OBNETCRE']

In [ ]:
from lib import string_to_float, get_code_aggregat, label_aggregat, label_fonction, pad_ndept

In [ ]:
from importlib import reload

In [ ]:
reload(lib)

In [ ]:
# convertir colonnes de dépense en float
for col in ['SD', 'SC', 'OBNETDEB', 'OBNETCRE', 'OOBDEB', 'OOBCRE']:
    data[col] = data[col].apply(string_to_float)

# convertir compte en str
data['COMPTE'] = data['COMPTE'].apply(str)

In [45]:
# étiquetter les aggregats
data['code_aggregat'] = data['COMPTE'].apply(get_code_aggregat)
data = data.dropna(subset=['code_aggregat'])
data['aggregat'] = data['code_aggregat'].apply(get_label_aggregat)

In [46]:
# calculer les dépenses selon les aggrégats
calcul_depense_1 = data['code_aggregat'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
calcul_depense_2 = data['code_aggregat'].isin([7.0, 8.0, 9.0])
calcul_depense_3 = data['code_aggregat'].isin([10.0, 11.0])

data.loc[calcul_depense_1, 'depense'] = data['SD'] - data['SC']
data.loc[calcul_depense_2, 'depense'] = data['OBNETDEB'] - data['OBNETCRE'] - (data['OOBDEB'] - data['OOBCRE']) 
data.loc[calcul_depense_3, 'depense'] = data['OBNETDEB']

In [51]:
# étiquetter les fonctions
data['FONCTION'].fillna(0, inplace=True)
data['code_fonction'] = data['FONCTION'].apply(lambda x: str(x)[0])
data['fonction'] = data['code_fonction'].apply(lambda x: label_fonction(int(x)))

In [71]:
# concat departement et code insee 
data['departement'] = data['NDEPT'].apply(pad_ndept)
data['INSEE'].fillna(0, inplace=True)
data['code_commune'] = data['departement'] + data['INSEE'].apply(lambda x: str(int(x)))

In [74]:
# pivoting 
index_cols = ['siren', 'IDENT', 'fonction', 'code_commune']
pivot_table = (
    pd
    .pivot_table(data, values='depense', index=index_cols, columns='aggregat', aggfunc='sum', fill_value=0)
    .reset_index()
)

In [75]:
# import and join pop data
pop_data = models.population_data()
pivot_pop = pivot_table.merge(pop_data, how='left', on='code_commune')

In [ ]:
pivot_pop.to_csv('pivot_pop.csv')

In [78]:
pivot_pop_sample = pivot_pop[pivot_pop['code_commune'].apply(str).isin(['31555', '06088', '44109', '34172'])]

In [ ]:
pivot_pop_sample.to_csv('pivot_pop_sample.csv')